In [37]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [41]:
# load all world bank data
# columns: 
# Country Name -> Location, 
# Country Code -> SpatialDimValueCode , 
# Year, (ww_value), ExtraFeatures
df_arg = pd.read_csv("Agriculture/agr_land.csv")
df_cpi = pd.read_csv("CPI/ConsumerPriceIndex(2010=100).csv")
df_edu = pd.read_csv("edu/school_enrollment.csv")
df_employ = pd.read_csv("Employment/EmploymentRate.csv")
df_exchange = pd.read_csv("exchange_rate/exchange_to_usd.csv")
df_GDP = pd.read_csv("GDP/GDP_per_capita.csv")
df_green = pd.read_csv("greenhouse/gas_emission.csv")
df_income = pd.read_csv("income/income_per_capita.csv")
df_pop = pd.read_csv("population/pop.csv")
df_tax = pd.read_csv("tax/tax.csv")

In [42]:
# unique countries
countries = df_arg['Country Name'].unique()
num_countries = len(countries)
print(num_countries)
# unique years
print(df_arg.columns) # 1960-2020 => 61
yrs = 61

266
Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')


In [43]:
def one_wordbank_feature(df,country):
    return df.loc[df['Country Name'] == country, '1960':'2020'].values

In [44]:
# directly create dictionary
d = dict()
d["Location"] = np.repeat(list(df_arg['Country Name']),yrs)
d["SpatialDimValueCode"] = np.repeat(list(df_arg['Country Code']),yrs)
d["Year"] = np.tile(list(df_arg.columns)[4:],num_countries)
d["arg"] = np.concatenate([one_wordbank_feature(df_arg,c) for c in countries]).ravel()
d["cpi"] = np.concatenate([one_wordbank_feature(df_cpi,c) for c in countries]).ravel()
d["edu"] = np.concatenate([one_wordbank_feature(df_edu,c) for c in countries]).ravel()
d["employ"] = np.concatenate([one_wordbank_feature(df_employ,c) for c in countries]).ravel()
d["exchange"] = np.concatenate([one_wordbank_feature(df_exchange,c) for c in countries]).ravel()
d["gdp"] = np.concatenate([one_wordbank_feature(df_GDP,c) for c in countries]).ravel()
d["green"] = np.concatenate([one_wordbank_feature(df_green,c) for c in countries]).ravel()
d["income"] = np.concatenate([one_wordbank_feature(df_income,c) for c in countries]).ravel()
d["pop"] = np.concatenate([one_wordbank_feature(df_pop,c) for c in countries]).ravel()
d["tax"] =  np.concatenate([one_wordbank_feature(df_tax,c) for c in countries]).ravel()

In [2]:
#print(d)
#assert(len(d["Location"]) == len(d["Year"]))
#assert(len(d["arg"]) == len(d["Year"]))

In [45]:
df_wb = pd.DataFrame(d)
#df_wb.to_csv("wb.csv",index=False)

In [46]:
def happy_feature(countries, years, features):
    res = []
    for country in tqdm(countries):
        for year in years:
            try:
                entry = df_happy.loc[((df_happy['Country name'] == country) 
                                      & (df_happy['year'] == int(year))),
                                     features].values[0]
                res.append(entry)
            except:
                entry = np.nan
                res.append(entry)
    return res
    
# merge happiness table
df_happy = pd.read_csv('Happiness/WHR2021.csv')
count_hap = df_happy['Country name'].unique()
count_hap_num = len(count_hap)
dh = dict()
dh['Location'] = np.repeat(count_hap,yrs)
dh['Year'] = np.tile(list(df_arg.columns)[4:],count_hap_num)
dh['LifeLadder'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Life Ladder')
dh['LogGDPPerCapita'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Log GDP per capita')
dh['SocialSupport'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Social support')
dh['Expectancy'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Healthy life expectancy at birth')
dh['Freedom'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Freedom to make life choices')
dh['Generosity'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Generosity')
dh['Corruption'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Perceptions of corruption')
dh['PositiveAffect'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Positive affect')
dh['NegativeAffect'] = happy_feature(count_hap,list(df_arg.columns)[4:],'Negative affect')

100%|██████████| 166/166 [00:12<00:00, 13.41it/s]


In [1]:
#print(dh)
#assert(len(dh["LifeLadder"]) == len(dh["Year"]))

In [52]:
dff_happy = pd.DataFrame(dh)
df_wbb = df_wb.merge(dff_happy, how='left')
df_wbb.to_csv("extra.csv",index=False)

In [106]:
df_wbb.head()

,Location,SpatialDimValueCode,Year,arg,cpi,edu,employ,exchange,gdp,green,...,tax,LifeLadder,LogGDPPerCapita,SocialSupport,Expectancy,Freedom,Generosity,Corruption,PositiveAffect,NegativeAffect
0,Aruba,ABW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,1961,11.111111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aruba,ABW,1962,11.111111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,ABW,1963,11.111111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,ABW,1964,11.111111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df_ww = pd.read_csv("ProvidedDatasets/tobacco_use_ww.csv")
# group by Location  
df_wbb['Year'] = df_wbb['Year'].astype(str)
df_ww['Year'] = df_ww['Year'].astype(str)
df_www = df_ww.merge(df_wbb,how='left')
df_www.to_csv("extra_ww.csv",index=False)

## EDA

In [32]:
df_www = pd.read_csv("extra_ww.csv")
df_www.head()

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Year,Gender,Value,arg,cpi,edu,...,tax,LifeLadder,LogGDPPerCapita,SocialSupport,Expectancy,Freedom,Generosity,Corruption,PositiveAffect,NegativeAffect
0,SEAR,South-East Asia,PRK,Democratic People's Republic of Korea,2018,Female,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUR,Europe,AZE,Azerbaijan,2018,Female,0.2,57.825393,152.900000,94.480080,...,12.971272,5.168,9.562,0.781,65.5,0.772,-0.232,0.561,0.593,0.191
2,AFR,Africa,ERI,Eritrea,2018,Female,0.3,75.168317,NaN,47.697109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EMR,Eastern Mediterranean,EGY,Egypt,2018,Female,0.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFR,Africa,GHA,Ghana,2018,Female,0.4,64.967654,231.807235,71.319687,...,12.616039,5.004,8.555,0.761,57.2,0.817,0.062,0.846,0.747,0.250


In [33]:
import plotly

In [34]:
import plotly.graph_objects as go
import plotly.express as px


'''fig1 = px.line(df_www, x="Year", y="green", 
               color='Location', title='annual greenhouse gas', markers=True)
fig1.show()

fig2 = px.line(df_www, x="Year", y="cpi", 
               color='Location', title='annual Consumer Price Index', markers=True)
fig2.show()

fig3 = px.line(df_www, x="Year", y="employ", 
               color='Location', title='annual school enrollment rate', markers=True)
fig3.show()

fig3 = px.line(df_www, x="Year", y="gdp", 
               color='Location', title='annual school gdp per capita', markers=True)
fig3.show()'''

fig3 = px.line(df_www, x="Year", y="LifeLadder", 
               color='Location', title='happiness', markers=True)
fig3.show()


In [35]:
row,col = df_www.shape
print(df_www.isna().sum()/row)
df_www.describe()


ParentLocationCode     0.000000
ParentLocation         0.000000
SpatialDimValueCode    0.000000
Location               0.000000
Year                   0.000000
Gender                 0.000000
Value                  0.000000
arg                    0.122297
cpi                    0.183445
edu                    0.358688
employ                 0.422073
exchange               0.255034
gdp                    0.127517
green                  0.120805
income                 0.170768
pop                    0.125280
tax                    0.378822
LifeLadder             0.457867
LogGDPPerCapita        0.457867
SocialSupport          0.463087
Expectancy             0.457867
Freedom                0.468307
Generosity             0.479493
Corruption             0.486204
PositiveAffect         0.466070
NegativeAffect         0.463833
dtype: float64


,Year,Value,arg,cpi,edu,employ,exchange,gdp,green,income,...,tax,LifeLadder,LogGDPPerCapita,SocialSupport,Expectancy,Freedom,Generosity,Corruption,PositiveAffect,NegativeAffect
count,4023.000000,4023.000000,3531.000000,3285.000000,2580.000000,2325.000000,2997.000000,3510.000000,3.537000e+03,3336.000000,...,2499.000000,2181.000000,2181.000000,2160.000000,2181.000000,2139.000000,2094.000000,2067.000000,2148.000000,2157.000000
mean,2012.000000,24.807159,41.582934,112.075145,86.508486,56.878077,478.605236,14399.129100,2.426587e+05,11388.558477,...,18.168774,5.555164,9.513971,0.819067,64.475873,0.769041,0.005633,0.730441,0.717851,0.270965
std,5.657557,15.745702,21.359801,40.345453,30.008470,10.748094,1443.963331,19629.350747,9.945794e+05,15065.166420,...,8.147519,1.166266,1.180462,0.117282,7.265953,0.133016,0.170545,0.203202,0.104378,0.082741
min,2000.000000,0.000000,0.930889,14.991313,6.197350,18.320000,0.044468,99.744920,2.000000e+01,84.451837,...,0.043495,2.702000,6.635000,0.434000,32.300000,0.304000,-0.335000,0.047000,0.421000,0.103000
25%,2010.000000,12.600000,25.273088,100.000000,69.107813,50.660000,2.701899,1512.126989,1.029000e+04,1267.380191,...,13.032967,4.607000,8.555000,0.750750,59.300000,0.687000,-0.117000,0.664000,0.638750,0.209000
50%,2014.000000,23.900000,42.857299,108.846334,95.352978,56.560000,15.375000,5506.820723,4.367000e+04,4429.395435,...,17.443762,5.526000,9.699000,0.852000,66.100000,0.789000,-0.024000,0.800000,0.734000,0.261000
75%,2016.000000,34.600000,58.063901,121.558878,104.496511,62.900000,222.656086,18098.908540,1.263800e+05,14185.895722,...,22.819233,6.467000,10.529000,0.913000,70.500000,0.882000,0.096000,0.867000,0.804250,0.321000
max,2018.000000,91.000000,85.464518,481.390314,163.934723,93.970000,14236.938770,118823.648400,1.235524e+07,80639.387780,...,110.182883,8.019000,11.645000,0.987000,76.800000,0.985000,0.698000,0.983000,0.944000,0.591000


In [36]:
df_www.groupby('ParentLocation')
fig = px.line(df_www.get_group('Asia'), x="Year", y="LifeLadder", 
               color='Location', title='happiness', markers=True)
fig.show()

AttributeError: 'DataFrame' object has no attribute 'get_group'